<a href="https://colab.research.google.com/github/SerhiiAndreiko/HW_DATA_10/blob/main/HW_DATA_10_second.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Частина 2**

​

В цій частині ми знову будемо працювати з датасетом fasion_mnist.



На відміну від попереднього завдання вам пропонується створити згорткову нейромережу, що використовує VGG16 в якості згорткової основи.



Навчіть отриману мережу на даних із датасету fasion_mnist. Спробуйте досягти максимально можливої точності класифікації за рахунок маніпуляції параметрами мережі. Під час навчання використовуйте прийоми донавчання та виділення ознак.



Порівняйте точність отриманої згорткової мережі з точністю багатошарової мережі з попереднього завдання. Зробіть висновки.

In [ ]:
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from skimage import color

# Завантаження даних Fashion MNIST
from tensorflow.keras.datasets import fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Додавання додаткового каналу
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

# Конвертація в колірне зображення (3 канали)
train_images = np.repeat(train_images, 3, axis=-1)
test_images = np.repeat(test_images, 3, axis=-1)

# Нормалізація даних
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Завантаження моделі VGG16 без верхніх повнозв'язних шарів
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(28, 28))

# Замороження ваг моделі VGG16
vgg_base.trainable = False

# Побудова згорткової мережі
model = Sequential([
    vgg_base,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Компіляція моделі
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Навчання моделі
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
history_vgg = model.fit(train_images, train_labels, epochs=20, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Оцінка моделі на тестових даних
test_loss_vgg, test_acc_vgg = model.evaluate(test_images, test_labels)
print('Точність на тестових даних VGG16:', test_acc_vgg)


ValueError: `input_shape` must be a tuple of three integers.